# Map Generator

The Map Generator provides GMs with a way to quickly generate a map that can be incorporated into their campaign.  Simply provide the notebook with a concept and the notebook will do the rest.

> [!IMPORTANT]
> You will need an [.NET 8 SDK](https://dotnet.microsoft.com/en-us/download/dotnet/8.0) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .NET Interactive.
> 
>
> To run the LLM prompts and semantic functions, make sure you have an
> - [Azure OpenAI Service Key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=rest-api) or
> - [OpenAI API Key](https://platform.openai.com).

## Step 1 - Define your Concept

Provide your testing context information in `input/user_message.txt`.

In [13]:
// Load all text in ../input/user_message.text
string user_message = await System.IO.File.ReadAllTextAsync("input/user_message.txt");

// Load all text in ../input/system_message.text
string system_message = await System.IO.File.ReadAllTextAsync("input/system_message.txt");


## Step 2 - Initial Configuration

Run the following cells to configure your AI generative settings.


Choose whether you wish to use the OpenAI or Azure OpenAI service.

In [14]:
bool useAzureOpenAI = false;

Execute the following code which will ask a few questions and save the settings to a local
`settings.json` configuration file, under the [config](config) folder. You can
also edit the file manually if you prefer. **Please keep the file safe.**

In [15]:
#!import config/Settings.cs

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

// Uncomment this if you're using OpenAI and need to set the Org Id
await Settings.AskOrg(useAzureOpenAI);

Settings: OK: AI model configured [config/settings.json]
Settings: OK: API key configured [config/settings.json]


## Step 3 - Instantiate Services

In [16]:
// Remove previously installed nuget packages
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: System.Numerics.Tensors"
#r "nuget: SkiaSharp"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using System.ComponentModel;
using System.Numerics.Tensors;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.TextToImage;

#pragma warning disable SKEXP0001, SKEXP0010

var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var httpClient = new HttpClient
{
    Timeout = TimeSpan.FromMinutes(10),
};


if (useAzureOpenAI)
{
    builder.AddAzureOpenAITextEmbeddingGeneration("text-embedding-ada-002", azureEndpoint, apiKey);
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey, httpClient: httpClient);
    builder.AddAzureOpenAITextToImage("dall-e-3", azureEndpoint, apiKey);
}
    
else
{
    builder.AddOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey, orgId);
    builder.AddOpenAIChatCompletion(model, apiKey, orgId, httpClient: httpClient);
    builder.AddOpenAITextToImage(apiKey, orgId);
}

var kernel = builder.Build();

// Create the output directory
var outputDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "output");
if (!Directory.Exists(outputDirectory))
{
    Directory.CreateDirectory(outputDirectory);
}

// Get AI service instance used to generate images
var dallE = kernel.GetRequiredService<ITextToImageService>();

// Get AI service instance used to extract embedding from a text
var textEmbedding = kernel.GetRequiredService<ITextEmbeddingGenerationService>();


Installed Packages Microsoft.SemanticKernel, 1.16.2 SkiaSharp, 2.88.8 System.Numerics.Tensors, 8.0.0

## Step 4 - Generate the Content

All generated output will be found in the `output` folder.  If no files exist in this folder post execution, expand this section and review the code execution results for failure.

In [17]:
#pragma warning disable SKEXP0001

// Create an httpClient
var httpClient = new HttpClient();

var imageUrl = await dallE.GenerateImageAsync(system_message + " " + user_message, 1024, 1024);

// Download the image to a local file
using (Stream stream = await httpClient.GetStreamAsync(imageUrl))
using (MemoryStream memStream = new MemoryStream())
{
    await stream.CopyToAsync(memStream);
    File.WriteAllBytes(Path.Combine(outputDirectory, $"map.png"), memStream.ToArray());
};